## RUN - Nuclear NICD Measurement

### Prep

In [ ]:
# Imports

import os, csv

import numpy as np
import scipy.ndimage as ndi

from skimage import io
from skimage.filters import threshold_otsu
from skimage.morphology import disk

In [ ]:
# Parameters

img_dir = r'..\Data\Images\HisGFP'
out_dir = r'..\Data\Measurements\HisGFP'

camera_noise = 10    # Max intensity to still be considered "dark current"
min_line_pxls = 100  # Min non-"dark current" pixels per line to not be cropped away

In [ ]:
# Find all relevant dirs and files

# Get all file paths
walk = os.walk(img_dir)

# Create a dict for all samples
path_dict = dict()
for w in walk:
    for f in w[-1]:
        if f.endswith('mch-NICD-import_every min for 10 mins.tif'):
            path_dict[w[0]] = dict()

# Parse out relevant files
for dpath in path_dict:
    for f in os.listdir(dpath):
        if f.endswith('-His-gfp timeseries_zstack evry 30 s for 10 min.tif'):
            path_dict[dpath]['nuc_on'] = f
        elif f.endswith('-Final His-gfp.tif'):
            path_dict[dpath]['nuc_off'] = f
        elif f.endswith('-mch-NICD-import_every min for 10 mins.tif'):
            path_dict[dpath]['nicd_on'] = f
        elif f.endswith('-Final mch-NICD.tif'):
            path_dict[dpath]['nicd_off'] = f
    if len(path_dict[dpath]) != 4:
        raise Exception("Incorrect number of files found for this path!")
        
# Report
for dpath in path_dict:
    print(dpath)
    for fkey in path_dict[dpath]:
        print('    {:<8}'.format(fkey), '--', path_dict[dpath][fkey])

In [ ]:
# Retrieve the relevant stack slice index

target_slice_dict = dict()
for dpath in path_dict:
    
    if '5min export' in dpath:
        slice_ann_fpath = os.path.join(img_dir, r'5min export/matched_planes.txt')
    if '10min export' in dpath:
        slice_ann_fpath = os.path.join(img_dir, r'10min export/matched_planes.txt')

    slice_ann_sample = int(path_dict[dpath]['nicd_on'].split('-')[0][:-1])
         
    slice_ann = np.loadtxt(slice_ann_fpath, skiprows=1, delimiter=',', dtype=int)
    target_slice = slice_ann[slice_ann_sample-1, 1] - 1
    
    target_slice_dict[dpath] = target_slice
    
# Check
for dpath in target_slice_dict:
    print(dpath, '--', target_slice_dict[dpath])

### Helper Function

In [ ]:
def crop_camera(img, ref_img, camera_noise, min_line_pxls):
    
    if ref_img.ndim == 2:
        crop_mask = np.sum(ref_img > camera_noise, axis=1) >= min_line_pxls
    elif ref_img.ndim == 3:
        crop_mask = np.sum(ref_img[0] > camera_noise, axis=1) >= min_line_pxls
    else:
        raise Exception("Unanticipated dimensionality of ref_img:", ref_img.ndim)
        
    img = img[..., crop_mask, :]
    
    return img

### Pipeline Function

In [ ]:
def run_pipeline(dpath, fnames, target_slice, camera_noise=10, min_line_pxls=100):
    
    # Report
    print("\nWorking on:", dpath)
    
    # Load data
    data = {}
    for fkey in fnames:
        img = io.imread(os.path.join(dpath, fnames[fkey]))
        data[fkey] = img
    
    # Select relevant data
    data['nuc_on']  = data['nuc_on'][::2, target_slice, :,:]
    data['nuc_off'] = data['nuc_off'][0, target_slice, :,:]    
    
    # For some samples, there is a mismatch in length of nuc_on and nicd_on..
    if data['nicd_on'].shape[0] > data['nuc_on'].shape[0]:
        data['nicd_on'] = data['nicd_on'][:-1]
    
    # Crop top and bottom
    ref_img = data['nuc_on'].copy()
    print("-- (y,x) cropped from", ref_img[0].shape, end=' ')
    for fkey in data:
        data[fkey] = crop_camera(data[fkey], ref_img, camera_noise, min_line_pxls)
    print("to", data['nuc_on'][0].shape)
    
    # Report what has been loaded
    print("-- Loaded and prepped arrays have these shapes:")
    for dkey in data:
        print("   ", dkey+':', data[dkey].shape)
    
    # Gaussian smoothing
    data['nuc_on_smooth'] = np.array([ndi.gaussian_filter(data['nuc_on'][t], 2) 
                                      for t in range(data['nuc_on'].shape[0])])
    data['nuc_off_smooth'] = ndi.gaussian_filter(data['nuc_off'], 2)
    
    # Otsu thresholding
    threshold = threshold_otsu(data['nuc_on_smooth'].flatten())
    data['nuc_on_mask']  = data['nuc_on_smooth'] > threshold
    data['nuc_off_mask'] = data['nuc_off_smooth'] > threshold
    
    # Getting whole cells (incl. cytoplasm)
    data['all_on_mask'] = np.array([ndi.binary_dilation(data['nuc_on_mask'][t], 
                                                        structure=disk(10, dtype=np.bool))
                                    for t in range(data['nuc_on'].shape[0])])
    data['cyt_on_mask'] = data['all_on_mask'] ^ data['nuc_on_mask']
    data['all_off_mask'] = ndi.binary_dilation(data['nuc_off_mask'], 
                                               structure=disk(10, dtype=np.bool))
    data['cyt_off_mask'] = data['all_off_mask'] ^ data['nuc_off_mask']
    print("-- Completed image processing steps")
    
    # Save resulting segmentation (for QC)
    qc_out_on = np.array([data['nuc_on_mask'], data['nuc_on'], data['nicd_on']])
    qc_out_on = np.moveaxis(qc_out_on, [0,1], [1,0])
    qc_out_off = np.array([data['nuc_off_mask'], data['nuc_off'], data['nicd_off']])
    io.imsave(os.path.join(dpath, 'QC_segmentation_ON.tif'), qc_out_on)
    io.imsave(os.path.join(dpath, 'QC_segmentation_OFF.tif'), qc_out_off)
    print("-- Saved segmentations for QC")

    # Get more conservative masks to avoid misattribution
    data['nuc_on_mask_cons'] = np.array([ndi.binary_erosion(data['nuc_on_mask'][t], 
                                                            structure=disk(5, dtype=np.bool))
                                         for t in range(data['nuc_on'].shape[0])])
    data['nuc_off_mask_cons'] = ndi.binary_erosion(data['nuc_off_mask'], 
                                                   structure=disk(5, dtype=np.bool))
    data['nuc_on_mask_dilated'] = np.array([ndi.binary_dilation(data['nuc_on_mask'][t], 
                                                                structure=disk(3, dtype=np.bool))
                                            for t in range(data['nuc_on'].shape[0])])
    data['cyt_on_mask_cons'] = data['all_on_mask'] ^ data['nuc_on_mask_dilated']
    data['nuc_off_mask_dilated'] = ndi.binary_dilation(data['nuc_off_mask'], 
                                                       structure=disk(3, dtype=np.bool))
    data['cyt_off_mask_cons'] = data['all_off_mask'] ^ data['nuc_off_mask_dilated']
    
    # Get `on` measurements over time
    m = measurements = dict()
    measurements['nuc_on_nicd_sum']  = [data['nicd_on'][t][data['nuc_on_mask'][t]].sum() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['nuc_on_nicd_mean'] = [data['nicd_on'][t][data['nuc_on_mask_cons'][t]].mean() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['nuc_on_area']      = data['nuc_on_mask'].sum(axis=(-1, -2))
    
    measurements['cyt_on_nicd_sum']  = [data['nicd_on'][t][data['cyt_on_mask'][t]].sum() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['cyt_on_nicd_mean'] = [data['nicd_on'][t][data['cyt_on_mask_cons'][t]].mean() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['cyt_on_area']      = data['cyt_on_mask'].sum(axis=(-1, -2))

    measurements['all_on_nicd_sum']  = [data['nicd_on'][t][data['all_on_mask'][t]].sum() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['all_on_nicd_mean'] = [data['nicd_on'][t][data['all_on_mask'][t]].mean() 
                                        for t in range(data['nuc_on'].shape[0])]
    measurements['all_on_area']      = data['all_on_mask'].sum(axis=(-1, -2))
    
    # Get `off` measurements
    measurements['nuc_off_nicd_sum']  = data['nicd_off'][data['nuc_off_mask']].sum()
    measurements['nuc_off_nicd_mean'] = data['nicd_off'][data['nuc_off_mask_cons']].mean()
    measurements['nuc_off_area']      = data['nuc_off_mask'].sum()

    measurements['cyt_off_nicd_sum']  = data['nicd_off'][data['cyt_off_mask']].sum()
    measurements['cyt_off_nicd_mean'] = data['nicd_off'][data['cyt_off_mask_cons']].mean()
    measurements['cyt_off_area']      = data['cyt_off_mask'].sum()

    measurements['all_off_nicd_sum']  = data['nicd_off'][data['all_off_mask']].sum()
    measurements['all_off_nicd_mean'] = data['nicd_off'][data['all_off_mask']].mean()
    measurements['all_off_area']      = data['all_off_mask'].sum()

    # Convert lists to arrays
    for mkey in measurements:
        if isinstance(m[mkey], list):
            m[mkey] = np.array(m[mkey])
    
    # Prepare for saving
    sample_number = fnames['nicd_on'].split('-')[0][:-1]
    fname_midfix = '_NICDquant_'
    dpath_out = dpath.replace(os.path.relpath(img_dir), os.path.relpath(out_dir))
    if not os.path.isdir(dpath_out):
        os.makedirs(dpath_out)
    
    # Save results for ON time course
    with open(os.path.join(dpath_out, sample_number+fname_midfix+'ON.tsv'), 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        measurement_keys = [mkey for mkey in measurements.keys() if 'on' in mkey]
        writer.writerow(['time_step'] + measurement_keys)
        for t in range(len(measurements['nuc_on_nicd_sum'])):
            data = [t] + [m[mkey][t] for mkey in measurement_keys]
            writer.writerow(data)
              
    # Save results for OFF time point
    with open(os.path.join(dpath_out, sample_number+fname_midfix+'OFF.tsv'), 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        measurement_keys = [mkey for mkey in measurements.keys() if 'off' in mkey]
        writer.writerow(['time_step'] + measurement_keys)
        data = [0] + [m[mkey] for mkey in measurement_keys]
        writer.writerow(data)

    print("-- Finished!")

### Execution Loop

In [ ]:
for dpath in path_dict:
    run_pipeline(dpath, path_dict[dpath], target_slice_dict[dpath],
                 camera_noise=camera_noise, min_line_pxls=min_line_pxls)